```{contents}
```
## **Vector Store in LangGraph**

A **vector store** is a specialized database that stores **dense numerical representations (embeddings)** of data and enables **semantic retrieval** through similarity search.
In LangGraph, vector stores serve as the backbone for **long-term memory, knowledge grounding, tool augmentation, and agent reasoning**.

---

### **1. Why Vector Stores Exist**

Traditional databases retrieve data using **exact matches**.
LLM systems require retrieval by **meaning**.

| Requirement         | Traditional DB | Vector Store |
| ------------------- | -------------- | ------------ |
| Keyword match       | ✔              | ✖            |
| Semantic match      | ✖              | ✔            |
| Fuzzy retrieval     | ✖              | ✔            |
| Reasoning memory    | ✖              | ✔            |
| Knowledge grounding | ✖              | ✔            |

Vector stores solve the **semantic retrieval problem**.

---

### **2. Core Concept**

```
Raw Data ──Embedding Model──► Vector ──Stored in DB
                                     │
Query ──Embedding Model──► Query Vector ──Similarity Search──► Relevant Knowledge
```

Vectors capture **semantic meaning** in high-dimensional space.

---

### **3. Role of Vector Store in LangGraph**

In LangGraph, vector stores integrate with the **stateful execution engine**:

```
Graph State ⇄ Vector Store ⇄ LLM Reasoning ⇄ Tool Execution
```

They provide:

* **Long-term memory**
* **Context augmentation**
* **Agent knowledge grounding**
* **Experience replay**
* **Historical recall**

---

### **4. Where Vector Stores Fit in a LangGraph Workflow**

```
User Query
   ↓
Embed Query
   ↓
Retrieve Relevant Documents  ←── Vector Store
   ↓
Inject into State
   ↓
LLM Node
   ↓
Next Decision
```

---

### **5. Minimal Working Example**

```python
from langchain.embeddings import OpenAIEmbeddings
from langchain.vectorstores import FAISS
from langchain.schema import Document

docs = [Document(page_content="LangGraph manages stateful workflows.")]
embeddings = OpenAIEmbeddings()
db = FAISS.from_documents(docs, embeddings)

def retrieve(state):
    query = state["query"]
    results = db.similarity_search(query, k=3)
    return {"context": [r.page_content for r in results]}
```

Used inside a LangGraph node.

---

### **6. State Integration**

```python
class State(TypedDict):
    query: str
    context: list[str]
```

Vector results become part of the shared state for downstream reasoning.

---

### **7. Vector Store Variants**

| Type                               | Use Case              |
| ---------------------------------- | --------------------- |
| In-memory (FAISS)                  | Fast, local           |
| Cloud (Pinecone, Weaviate, Milvus) | Scalable              |
| Hybrid (Elastic + vectors)         | Structured + semantic |
| Personal memory store              | Agent memory          |
| Knowledge base                     | Enterprise documents  |

---

### **8. Retrieval Strategies**

| Strategy             | Description               |
| -------------------- | ------------------------- |
| k-NN                 | Top-k similar vectors     |
| MMR                  | Diversity-aware retrieval |
| Hybrid search        | Keyword + vector          |
| Metadata filtering   | Context constraints       |
| Time-aware retrieval | Recency bias              |
| Role-based retrieval | Security control          |

---

### **9. Advanced Patterns in LangGraph**

| Pattern                  | Purpose                         |
| ------------------------ | ------------------------------- |
| Memory Agent             | Persistent personal memory      |
| Knowledge Grounding      | Reduce hallucinations           |
| Tool-Augmented Retrieval | Combine tools + memory          |
| Experience Replay        | Learning from history           |
| Self-Improving Memory    | Update vector store dynamically |

---

### **10. Production Architecture**

```
LangGraph Runtime
      │
State Store ──► Vector Store ──► Embedding Model
      │                 │
Checkpoint DB       Knowledge DB
```

---

### **11. Why Vector Stores Are Critical for Agents**

Without vector stores:

* Agents forget
* No grounding
* High hallucination
* No learning

With vector stores:

* Persistent memory
* Context awareness
* Knowledge grounding
* Improved reasoning

---

### **12. Mental Model**

> **Vector store = long-term semantic memory of the LangGraph system**

It allows LangGraph to behave like a **learning, remembering, reasoning system**, not just a prompt pipeline.



### Demonstration

In [1]:
from typing import TypedDict, List
from langgraph.graph import StateGraph, END
from langchain_openai import OpenAIEmbeddings
from langchain_classic.vectorstores import FAISS
from langchain_classic.schema import Document

# -------------------- 1. Build Vector Store --------------------

docs = [
    Document(page_content="LangGraph enables stateful agent workflows."),
    Document(page_content="Vector stores provide long-term memory for LLMs."),
    Document(page_content="Cyclic graphs enable self-correction and planning."),
]

embeddings = OpenAIEmbeddings()
db = FAISS.from_documents(docs, embeddings)

# -------------------- 2. Define Graph State --------------------

class State(TypedDict):
    query: str
    context: List[str]
    answer: str

# -------------------- 3. Graph Nodes --------------------

def retrieve(state: State):
    results = db.similarity_search(state["query"], k=2)
    return {"context": [r.page_content for r in results]}

def reason(state: State):
    joined = " ".join(state["context"])
    return {"answer": f"Answer based on memory: {joined}"}

# -------------------- 4. Build LangGraph --------------------

builder = StateGraph(State)
builder.add_node("retrieve", retrieve)
builder.add_node("reason", reason)

builder.set_entry_point("retrieve")
builder.add_edge("retrieve", "reason")
builder.add_edge("reason", END)

graph = builder.compile()

# -------------------- 5. Run --------------------

result = graph.invoke({"query": "How does LangGraph help agents?"})
result


{'query': 'How does LangGraph help agents?',
 'context': ['LangGraph enables stateful agent workflows.',
  'Cyclic graphs enable self-correction and planning.'],
 'answer': 'Answer based on memory: LangGraph enables stateful agent workflows. Cyclic graphs enable self-correction and planning.'}